In [3]:
# загрузка 

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import json

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

path_json = '/Users/gleb/Desktop/internerLAB/internetlab/docs/clean_docs/clean_fed.json'

title = [i.capitalize() for i in load_data(path_json)["title"]]
content = load_data(path_json)["text"]

title[1]

'Концепция формирования информационного общества в россии'

In [4]:
# tf-idf

vectorizer = TfidfVectorizer(
    lowercase=True,
    max_features=100, # тут топ частотность, т.е от самого частого к менее
    max_df=0.8, # рейтинг слов больше 80% будут игнорироваться
    min_df=3, # слова которые появляются реже 3 раз - игнорируются
    ngram_range = (2,4), # n-gram от n слова до фразы из m
    stop_words = stopwords.words('russian')
)

vectors = vectorizer.fit_transform(content)

features_name = vectorizer.get_feature_names_out() 

dense = vectors.todense() # -> matrix
denselist = dense.tolist() # -> list-matrix


In [5]:
# загрузка k-w

# финальный список 
all_keywords = []

# в данном цикле проверяется частотность слова/фраз из вектора. Если из итоговый рейтинг > 0 то это k-words
for desc in denselist:
    x=0
    keywords = list(set([]))
    for word in desc:
        if word > 0:
            keywords.append(features_name[x])
        x +=1
    all_keywords.append(keywords)    
    
# # код который я не смог реализовать правильно, чтобы он считал k-w по всем документам (считает только по первому)

# счетчик k-w по каждому доку
# keyword_counts = {}

# # финальный список всех k-w по всем докам
# f_list = []

# for desc in denselist:
#     keywords_in_doc = set()
#     for ind, word in enumerate(desc):
#         if word > 0:
#             keyword = features_name[ind]
#             if keyword not in keywords_in_doc:
#                 keyword_counts[keyword] = keyword_counts.get(keyword, 0) + 1
#                 keywords_in_doc.add(keyword)
                
#     f_list.append(keyword_counts)
#     keywords_in_doc = {}

In [6]:
# кастомное решение поиска k-w с помощью обычного метода count() !рузультаты отличаются, они не совсем верные! 

def count_keyword_occurrences(content, all_keywords):
    keyword_counts = []
    
    for i, doc in enumerate(content):
        keywords = all_keywords[i]
        temp = []
        for keyword in keywords:
            counter = doc.lower().count(keyword)
            temp.append(counter)
        keyword_counts.append(temp)

    return keyword_counts

# здесь форматирую матрицу в обычный массив
kw_counter = count_keyword_occurrences(content, all_keywords)
kw_count = []
for i in kw_counter:
    for j in i:
        kw_count.append(j)

### df

In [7]:
# df по k-word 
data = [[title, kw] for title, content_list in zip(title, all_keywords) for kw in content_list]
df = pd.DataFrame(data, columns=['title', 'k-word'])
df['appearance']= kw_count

In [8]:
# df.iloc[95:105]
df.head(10)

,title,k-word,appearance
0,Стратегия развития информационного общества в ...,безопасности государства,2
1,Стратегия развития информационного общества в ...,государственного управления,5
2,Стратегия развития информационного общества в ...,государственной власти,12
3,Стратегия развития информационного общества в ...,государственных муниципальных,1
4,Стратегия развития информационного общества в ...,государственных муниципальных услуг,1
5,Стратегия развития информационного общества в ...,государственных органов,10
6,Стратегия развития информационного общества в ...,законодательства российской,2
7,Стратегия развития информационного общества в ...,законодательства российской федерации,2
8,Стратегия развития информационного общества в ...,защите прав,1
9,Стратегия развития информационного общества в ...,защиты информации,4


In [9]:
df.to_csv('k-w_table.csv', index=False, encoding='UTF-16')

In [21]:
# загрзка в док
with open ("/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/f_f_c_tf_results", "w", encoding="utf-8") as d:
    for i, j in enumerate(all_keywords):
        d.write('\n')
        d.write(f'- - - {title[i].upper()} - - - ')    
        d.write('\n')
        for w in range(len(j)):
            d.write(all_keywords[i][w])
            d.write('\n')